# Series Expansion Approximations

This notebook lets you play around with different ways of approximating analytic functions in a given interval.  The three ways are

1. Legendre series
2. Taylor series
3. Fourier series
    
You have previously seen Taylor series and Fourier series in your first-year courses, but the Legendre polynomials are new in this course on differential equations.

Taylor series are defined by
\begin{equation}
    T_f(x) = \sum_{k=0}^\infty \frac{f^{(k)}(a)}{k!} {(x-a)}^k
    \quad\text{where}\quad
    f^{(k)}(a) = \left.\frac{\mathrm{d}^k f(x)}{\mathrm{d}x^k}\right\lvert_{x=a}
\end{equation}
and Fourier series on the interval $[-1, 1)$ are defined by
\begin{equation}
    F_f(x) = \frac12 a_{f,0} + \sum_{k=1}^\infty \biggl(a_{f,k}\cos(n\pi x) + b_{f,k}\sin(n\pi x)\biggr),
\end{equation}
where the coefficients are
\begin{equation}
    a_{f,k} = \int_{-1}^1 f(x) \cos(k \pi x)\,\mathrm{d}x \quad\text{and}\quad b_{f,k} = \int_{-1}^1 f(x) \sin(k \pi x)\,\mathrm{d}x.
\end{equation}

## Initialisation code

These few cells produce some functions for finding (to a good approximation) the various series that we're interested in.  You don't need to understand the details of how any of this works, but feel free to ask.  The Legendre series and Fourier series calculations are done in the same way they are described above, but the Taylor series ones are not.

**You need to run all the cells in this section for anything to work.**

In [1]:
%matplotlib inline
from matplotlib import pyplot
import matplotlib
import scipy.special
import scipy.integrate
import numpy as np
import numpy.polynomial
import ipywidgets

In [2]:
def legendre_series(f, n):
    r"""
    Calculate the terms of the Legendre series expansion of the function
    ..math:`f(x)` with the first ..math:`n_terms` terms.  This will be
    the terms up to but _excluding_ the coefficient of ..math:`P_n(x)`.
    
    The resultant object can be called like a function to return the
    value of the approximation at values of ..math:`x`.
    """
    if n < 1:
        raise ValueError("'n' must be at least 1.")
    def integrand(x, k):
        return scipy.special.eval_legendre(k, x)*f(x)
    # Approximate the inner product integral for each of the polynomials,
    # including the normalisation factor.  `scipy.integrate.quad` performs
    # numerical integration (also called 'quadrature') until a particular
    # precision goal is reached.
    return np.polynomial.legendre.Legendre(np.array([
        scipy.integrate.quad(integrand, -1, 1, args=(k,))[0] * (k + 0.5)
        for k in range(n)
    ]))


def taylor_coefficient(f, k, a=15):
    r"""
    Calculate the ..math:`k`th coefficient in the Taylor expansion of
    ..math:`f(x)` around the point ..math:`x_0 = 0`.  The first term is
    ..math:`k = 0`, as this is the zeroth-order term.
    
    ``a`` is a precision factor, and should probably just be left as-is.
    """
    if k == 0:
        return f(0)
    # The standard way of defining Taylor series with derivatives
    # and factorials doesn't play nicely with numerical methods.  This
    # method is based on contour integration (magic).
    scale = np.exp(-a/k)
    return np.exp(a)/k * sum(
        (-1)**n * np.imag(f(scale * np.exp(1j*np.pi*(0.5-n)/k)))
        for n in range(1, k+1)
    )

def taylor_series(f, n, a=15):
    r"""
    Calculate the first ..math:`n` terms of the Taylor series
    expansion of ..math:`f(x)` around the point ..math:`x_0 = 0` up
    to but excluding the term ..math:`x^n`.
    
    The resultant object can be called like a function to return the
    value of the approximation at values of ..math:`x`.
    """
    if n < 1:
        raise ValueError("'n' must be at least 1.")
    return np.polynomial.Polynomial([
        taylor_coefficient(f, k, a)
        for k in range(n)
    ])


class fourier_series:
    r"""
    Calculate the first ..math:`n` terms of the Fourier series
    expansion of ..math:`f(x)` when mapped to the period
    ..math:`[-1, 1)`.
    
    The terms are "numbered" in the order
    ..math::
        a_0, b_1, a_1, b_2, a_2, \dotsc
    This is by analogy to Taylor series; the first term is the
    constant, then the lowest-order odd term, the next-lowest even
    term, and so on.
    
    The resultant object can be called like a function to return the
    value of the approximation at values of ..math:`x`.
    """
    def __init__(self, f, n):
        if n < 1:
            raise ValueError("'n' must be at least 1.")
        self._n_a = (n + 1) // 2
        self._n_b = n - self._n_a
        self.a = np.empty((self._n_a,), dtype=np.float64)
        # To keep the labelling clear I store the `b[0] = 0` too.
        self.b = np.empty((self._n_b + 1,), dtype=np.float64)
        self.a[0] = 0.5 * scipy.integrate.quad(f, -1, 1)[0]
        self.b[0] = 0
        def cosint(x, k): return f(x) * np.cos(k*np.pi*x)
        def sinint(x, k): return f(x) * np.sin(k*np.pi*x)
        for k in range(1, self._n_a):
            self.a[k] = scipy.integrate.quad(cosint, -1, 1, args=(k,))[0]
        for k in range(1, self._n_b):
            self.b[k] = scipy.integrate.quad(sinint, -1, 1, args=(k,))[0]

    def __call__(self, xs):
        out = np.zeros_like(xs)
        for k in range(self._n_a):
            out += self.a[k] * np.cos(k*np.pi * xs)
        for k in range(1, self._n_b):
            out += self.b[k] * np.sin(k*np.pi * xs)
        return out

In [3]:
def series_plot(f, order):
    r"""
    Plot a function ``f`` and its Legendre-, Taylor- and Fourier-series
    approximations of the given order on the interval ..math:`[-1, 1)`.
    """
    xs = np.linspace(-1, 1, 301)
    cm = matplotlib.cm.viridis
    _, axes = pyplot.subplots()
    fs = np.array([f(x) for x in xs])
    axes.plot(xs, fs, label="Exact", color='black', linewidth=2, dashes=(5, 5))
    axes.plot(xs, legendre_series(f, order)(xs), label="Legendre", color=cm(0))
    axes.plot(xs, taylor_series(f, order)(xs), label="Taylor", color=cm(0.4))
    axes.plot(xs, fourier_series(f, order)(xs), label="Fourier", color=cm(0.8))
    axes.set_xlim((xs[0] - 0.03*(xs[-1]-xs[0]), xs[-1] + 0.03*(xs[-1]-xs[0])))
    # Base the y axis on the actual function rather than all the data, so the
    # Taylor series doesn't blow out the entire scale when it's wrong.
    mn, mx = np.min(fs), np.max(fs)
    axes.set_ylim((mn - 0.1*(mx-mn), mx + 0.1*(mx-mn)))
    axes.legend()

## Investigating series behaviour at low orders

I have defined some mathematical functions in Python which are hopefully quite clear from the code.  Below them, I have plotted the function, and the series expansions of the function with a controllable `order` parameter.  Feel free to copy-paste to see what happens when you use your own functions.

Think about these questions, and then discuss them in your groups.

1. Which series would you call the "most accurate"?  Why?  Does it depend on the function?
2. What sorts of functions are the different expansions best at approximating?  Which are they bad at?
3. The Legendre series often seems to "give up" in the middle of some shapes at low orders (_e.g._ sinusoids).  Why is this, and are there any things the series is still useful for?
4. The Taylor series almost invariably has the largest pointwise error.  Why is this?  What is the Taylor series useful for?

In [4]:
def high_order_polynomial(x):
    return np.polynomial.Polynomial([
        -0.0372875, 0.674885, 1.34898, -12.652, -7.15369, 57.7268,
        8.73373, -104.258, 10.0257, 79.9955, -21.4594, -21.5587, 8.38861,
    ])(x)

ipywidgets.interact(
    lambda terms: series_plot(high_order_polynomial, terms),
    terms=(1, 15, 1),
);

interactive(children=(IntSlider(value=8, description='terms', max=15, min=1), Output()), _dom_classes=('widget…

In [5]:
def logistic(x):
    return 1 / (1 + np.exp(-5*x))

ipywidgets.interact(
    lambda terms: series_plot(logistic, terms),
    terms=(1, 12, 1),
);

interactive(children=(IntSlider(value=6, description='terms', max=12, min=1), Output()), _dom_classes=('widget…

In [6]:
def lorentzian(x, c=0.2):
    return c*np.pi / (c*np.pi + (x/c)**2)

ipywidgets.interact(
    lambda terms: series_plot(lorentzian, terms),
    terms=(1, 12, 1),
);

interactive(children=(IntSlider(value=6, description='terms', max=12, min=1), Output()), _dom_classes=('widget…